In [1]:
import sys
sys.path.append('../src')
from train_process.train_CNN import train
import pandas as pd

### 数据探索和可视化

In [ ]:
file_index = [i for i in range(1,34)]
for eeg_file_name in file_index:
    xlsx=  rf'../src/data/EEG_DATASET/{eeg_file_name}/{eeg_file_name}.xlsx'
    label_df = pd.read_excel(xlsx, 'Sheet1', header=None)
    label_df.rename(columns={0: 'label'}, inplace=True)
    # 打印label中0的个数和1的个数
    print(f"第{eeg_file_name}个人的数据中，label中0的个数为{label_df[label_df['label'] == 0].shape[0]}，label中1的个数为{label_df[label_df['label'] == 1].shape[0]}")


In [ ]:
import utils.data_preprocessing.temp_utils as utils
import matplotlib.pyplot as plt
import numpy as np
import mne

In [ ]:
eeg_file_name = 16
file_path = rf'../src/data/EEG_DATASET/{eeg_file_name}/{eeg_file_name}.edf'
# utils.read_edf(file_path)
# 绘制EEG信号
raw = utils.read_edf(file_path)
raw = utils.rename_edf(raw)

# 选取第1秒到第20秒的数据
start, stop = raw.time_as_index([1, 30])  # 返回第1秒和第20秒的索引

# 选取数据
data, times = raw[:, start:stop]

# 绘制信号
raw.plot(start=1, duration=29, scalings='auto')

In [ ]:
eeg_file_name = 1
xlsx=  rf'../src/data/EEG_DATASET/{eeg_file_name}/{eeg_file_name}.xlsx'
label_df = pd.read_excel(xlsx, 'Sheet1', header=None)
label_df.rename(columns={0: 'label'}, inplace=True)
# print(label_df[1:30])

### 模型训练

In [2]:
train_args = {
    "eeg_file_names": [str(i) for i in range(1,33)],  # 示例文件名
    "test_mode": False,
    "image_size": 32,
    "batch_size": 512,
    "train_size": 0.7,
    "val_size": 0.1,
    "test_size": 0.2,
    "test_on_each_patient": False,
    "learning_rate": 0.005,
    "out": "D:/Research/EEG/EEG_DL_Classifier/checkpoints",
    "wandb_run_name": "EEG_Classification_Run",
    "disable_wandb": False,
    "enable_progress_bar": True,
    "max_epochs": 30,  # 示例训练周期数
    "patience": 3,
}

train(train_args)


wandb: Currently logged in as: cretaceousmart. Use `wandb login --relogin` to force relogin


Jie Log: num_positive: 50576, num_negative: 64624
Jie Log: total_size: 115200
Jie Log: EEG_DataModule.setup()执行完毕： train_size: 80640, val_size: 11520, test_size: 23040


d:\Dev\Anaconda\envs\mne\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:639: Checkpoint directory D:\Research\EEG\EEG_DL_Classifier\checkpoints exists and is not empty.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

d:\Dev\Anaconda\envs\mne\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


...

Validation: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_accuracy          0.7983506917953491
         val_f1             0.7160457372665405
        val_loss            0.44492292404174805
      val_precision         0.9625886678695679
       val_recall           0.5706292390823364
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Jie Log: num_positive: 50576, num_negative: 64624
Jie Log: total_size: 115200
Jie Log: EEG_DataModule.setup()执行完毕： train_size: 80640, val_size: 11520, test_size: 23040


d:\Dev\Anaconda\envs\mne\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.8038628697395325
         test_f1            0.7193979620933533
        test_loss           0.43315136432647705
     test_precision         0.9643689393997192
       test_recall          0.5746467709541321
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [ ]:
import torch
from torchviz import make_dot

### 模型可视化

In [ ]:
import torchvision.models as models
from model.CNN import CNN
# resnet = models.resnet50(pretrained=True)

# x = torch.randn(1, 1, 128, 128)
x = torch.zeros(1, 1, 224, 224, dtype=torch.float, requires_grad=False)
model = CNN(learning_rate=0.05)
out = model(x)
make_dot(out)  # plot graph of variable, not of a nn.Module